## Overview

CPython -common implementation we all use-  does not use multiple CPUs by default.

Expect *±n-times* speed-up of execution time with *n cores*.

Each additional process will increase communication overhead and decrease available RAM up to a significant overall slowdown.

Amdahls law: *If only a small part of your code can be parallelized, it doesn't matter how many CPUs you use - it won't run much faster after all*

If each process needs to communicate with every other Python process, the communication overhead will slowly overwhelm the processing and slow things down.

As more and more processes are added, overalle performance can be slowed down.

Typlical jobs for multiprocessing module:
* Parallelize a CPU-bound task with Process or Pool bjects
* Parallelize a IO-bound task in a Pool with threads (dummy module)
* Share pickled work via a Queue
* Share state between parallelized workers (bytes, primitive datatypes, dicts, lists)

Threads in Python are bound by the GIL (Global Interpreter Lock), so only one thread may interact with Python projects at a time.

By using processes, we run a number of Python interpreters in parallel - each with a private memory space with its own GIL -and each running in series -> no competition for each GIL.

## Multiprocessing module

Main components:
* **Process**: 
forked copy of current process (new pid - independent child process in operating system - provide it with target method to run)
* **Pool**:
wraps the process/threading - Thread API into a convenient pool of workers that share chunk of work und return aggregated result
* **Queue**:
FIFO queue allowing multiple producers and consumers
* **Pipe**:
Uni-/bidirectional communication cannel between two processes
* **Manager**:
High level managed interface to share Python objects between processes
* **ctypes**:
Allows sharing of primitive datatypes (ints flaots, bytes) between processes after they have forked
